In [14]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [2]:
estudiantes_df = pd.read_csv('datos_estudiantes_1000.csv')

In [3]:
estudiantes_df.head()

,ID,Edad,Promedio_Notas,Asistencia_%,Asignaturas_Desaprobadas,Uso_Plataforma_Virtual,Participa_Tutorias,Trabaja,Abandona
0,1,19,9.3,100.7,1,0,0,0,1
1,2,29,15.4,80.1,0,1,0,1,1
2,3,19,13.5,70.1,1,0,1,0,0
3,4,23,13.4,80.3,1,0,1,0,1
4,5,18,13.6,85.9,4,0,1,1,0


# 4.1 Analisis de Datos Exploratorio Inicial

In [4]:
estudiantes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        1000 non-null   int64  
 1   Edad                      1000 non-null   int64  
 2   Promedio_Notas            1000 non-null   float64
 3   Asistencia_%              1000 non-null   float64
 4   Asignaturas_Desaprobadas  1000 non-null   int64  
 5   Uso_Plataforma_Virtual    1000 non-null   int64  
 6   Participa_Tutorias        1000 non-null   int64  
 7   Trabaja                   1000 non-null   int64  
 8   Abandona                  1000 non-null   int64  
dtypes: float64(2), int64(7)
memory usage: 70.4 KB


In [5]:
estudiantes_df.describe()

,ID,Edad,Promedio_Notas,Asistencia_%,Asignaturas_Desaprobadas,Uso_Plataforma_Virtual,Participa_Tutorias,Trabaja,Abandona
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.00000
mean,500.500000,22.819000,13.520500,85.467300,1.52400,0.485000,0.495000,0.499000,0.63400
std,288.819436,3.661372,2.029845,9.829633,1.22185,0.500025,0.500225,0.500249,0.48195
min,1.000000,17.000000,7.000000,53.300000,0.00000,0.000000,0.000000,0.000000,0.00000
25%,250.750000,20.000000,12.175000,78.600000,1.00000,0.000000,0.000000,0.000000,0.00000
50%,500.500000,23.000000,13.500000,85.600000,1.00000,0.000000,0.000000,0.000000,1.00000
75%,750.250000,26.000000,15.000000,92.425000,2.00000,1.000000,1.000000,1.000000,1.00000
max,1000.000000,29.000000,20.200000,117.400000,7.00000,1.000000,1.000000,1.000000,1.00000


**Se ve que hay filas que marcan asistencia mayor a 100%. Se reemplazará por 100%(el maximo valor posbile)**

In [6]:
estudiantes_df.loc[ estudiantes_df['Asistencia_%'] > 100,'Asistencia_%'] = 0

In [7]:
estudiantes_df.describe()

,ID,Edad,Promedio_Notas,Asistencia_%,Asignaturas_Desaprobadas,Uso_Plataforma_Virtual,Participa_Tutorias,Trabaja,Abandona
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.00000
mean,500.500000,22.819000,13.520500,78.405500,1.52400,0.485000,0.495000,0.499000,0.63400
std,288.819436,3.661372,2.029845,22.806286,1.22185,0.500025,0.500225,0.500249,0.48195
min,1.000000,17.000000,7.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
25%,250.750000,20.000000,12.175000,76.075000,1.00000,0.000000,0.000000,0.000000,0.00000
50%,500.500000,23.000000,13.500000,83.800000,1.00000,0.000000,0.000000,0.000000,1.00000
75%,750.250000,26.000000,15.000000,90.400000,2.00000,1.000000,1.000000,1.000000,1.00000
max,1000.000000,29.000000,20.200000,100.000000,7.00000,1.000000,1.000000,1.000000,1.00000


# 4.2 Definición del Problema

In [8]:
variables_prediccion = ['Edad', 'Promedio_Notas', 'Asistencia_%', 'Asignaturas_Desaprobadas', 'Uso_Plataforma_Virtual', 'Participa_Tutorias', 'Trabaja']
datos_modelado = estudiantes_df[variables_prediccion]
datos_modelado

,Edad,Promedio_Notas,Asistencia_%,Asignaturas_Desaprobadas,Uso_Plataforma_Virtual,Participa_Tutorias,Trabaja
0,19,9.3,0.0,1,0,0,0
1,29,15.4,80.1,0,1,0,1
2,19,13.5,70.1,1,0,1,0
3,23,13.4,80.3,1,0,1,0
4,18,13.6,85.9,4,0,1,1
...,...,...,...,...,...,...,...
995,22,12.7,83.3,1,0,0,0
996,21,12.3,90.5,1,1,1,1
997,21,13.3,81.0,3,0,1,0
998,24,13.2,90.8,1,0,1,0


Se identifican las variables que estan relacionadas a la variable objetivo, estas son : **'Edad', 'Promedio_Notas', 'Asistencia_%', 'Asignaturas_Desaprobadas', 'Uso_Plataforma_Virtual', 'Participa_Tutorias', 'Trabaja'**

# 4.3 Modelado con Scikit-learn

In [9]:
parametros = datos_modelado.to_numpy()
etiquetas = np.where( estudiantes_df.Abandona.to_numpy() == 1, 'abandona', 'no abandona')

In [10]:
def calcular_precision(matriz_confucion):
    tn, fp, fn, tp = matriz_confucion.ravel()
    return tp * 100 / (tp + fp)

def calcular_sensibilidad(matriz_confucion):
    tn, fp, fn, tp = matriz_confucion.ravel()
    return tp * 100 / (tp + fn)

def calcular_especificidad(matriz_confucion):
    tn, fp, fn, tp = matriz_confucion.ravel()
    return tn * 100 / (tn + fp)

def evaluar_modelo(algoritmo, parametros_array, etiquetas_array):
    clasificador = algoritmo
    #Dividir dataset enre etrenamiento y tests
    parametros_entrenamiento, parametros_test, etiquetas_entrenamiento, etiquetas_test = train_test_split(parametros_array, etiquetas_array)
    clasificador.fit(parametros_entrenamiento,etiquetas_entrenamiento)
    prediccion = clasificador.predict(parametros_test)
    #Calcular efectividad del modelo
    matriz_con = confusion_matrix(etiquetas_test, prediccion,labels=['no abandona', 'abandona'])
    precision = calcular_precision(matriz_con)
    sensibilidad = calcular_sensibilidad(matriz_con)
    especificidad = calcular_especificidad(matriz_con)

    print(f'Precisión del modelo {clasificador} : {precision:.2f}%')
    print(f'Sensibilidad del modelo {clasificador} : {sensibilidad:.2f}%')
    print(f'Especificidad del modelo {clasificador} : {especificidad:.2f}%')
    

In [11]:
evaluar_modelo(KNeighborsClassifier(n_neighbors=7), parametros, etiquetas)

Precisión del modelo KNeighborsClassifier(n_neighbors=7) : 63.28%
Sensibilidad del modelo KNeighborsClassifier(n_neighbors=7) : 71.34%
Especificidad del modelo KNeighborsClassifier(n_neighbors=7) : 30.11%


In [12]:
evaluar_modelo(DecisionTreeClassifier(), parametros, etiquetas)

Precisión del modelo DecisionTreeClassifier() : 69.39%
Sensibilidad del modelo DecisionTreeClassifier() : 66.23%
Especificidad del modelo DecisionTreeClassifier() : 53.12%


In [13]:
evaluar_modelo(LogisticRegression(max_iter=1000), parametros, etiquetas)

Precisión del modelo LogisticRegression(max_iter=1000) : 72.55%
Sensibilidad del modelo LogisticRegression(max_iter=1000) : 91.93%
Especificidad del modelo LogisticRegression(max_iter=1000) : 37.08%


### Modelo Simple de Red Neuronal Perceptron
**Usando 100 000 neruonas**

In [19]:
nn_model = MLPClassifier(hidden_layer_sizes=(100000,), 
                         activation='relu', 
                         max_iter=1000, 
                         random_state=42)

evaluar_modelo(nn_model, parametros, etiquetas)

Precisión del modelo MLPClassifier(hidden_layer_sizes=(100000,), max_iter=1000, random_state=42) : 74.50%
Sensibilidad del modelo MLPClassifier(hidden_layer_sizes=(100000,), max_iter=1000, random_state=42) : 89.22%
Especificidad del modelo MLPClassifier(hidden_layer_sizes=(100000,), max_iter=1000, random_state=42) : 38.55%


# 4.5 Discusion de Resultados
Los modelos que resularaon con mejor precicion, sensibilidad y especifidad fueron Regresion Logaritmica usando 1000 iteraciones maximas, Red Neuronal Perceptron usando 100000 neuronas